In [5]:
# Helper functions to convert latlng
# from https://stackoverflow.com/questions/33997361/how-to-convert-degree-minute-second-to-degree-decimal

import re

def dms2dd(direction, degrees, minutes, seconds):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[º°\'",]+', dms)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    

    return (lat)

# Temple

In [44]:
import csv, re

def parse_csv (infile, outfile):

    with open(infile) as csvfile:
        reader = csv.reader(csvfile)

        next(reader, None)

        outlist = []

        for row in reader:
            if row[0].isnumeric():
                buffer = {
                    "name_indo": "",
                    "name_hanzi": "",
                    "name_pinyin": "",
                    "neighbourhood": "",
                    "gps_lat": "",
                    "gps_lon": "",
                    "address": "",
                    "ref_no": ""
                }
                
                buffer["name_indo"] = row[1].strip()
                buffer["name_hanzi"] = row[2].strip()
                lat = row[3].strip() + "," + row[4].strip() + "." + row[5].strip()
                lat = lat.replace(" ", ",")
                lat = lat.replace(",,", ",")

                buffer["gps_lat"] = parse_dms(lat)

                address = row[6]
                
                buffer["ref_no"] = row[7]
            else:
                buffer["neighbourhood"] = row[1].strip()
                buffer["name_pinyin"] = row[2].strip()
                
                lon = row[3].strip() + "," + row[4].strip() + "." + row[5].strip()
                lon = lon.replace(" ", ",")
                lon = lon.replace(",,", ",")
                buffer["gps_lon"] = parse_dms(lon)

                address = address.strip() + ", "+ row[6].strip()
                buffer["address"] = address

                outlist.append(buffer)

        keys = outlist[0].keys()
        with open(outfile, 'w', newline='', encoding='utf-8')  as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(outlist)

In [48]:
parse_csv("raw_barat.csv", "1_West.csv")
parse_csv("raw_tengah.csv", "1_Central.csv")
parse_csv("raw_selatan.csv", "1_South.csv")
parse_csv("raw_timur.csv", "1_East.csv")
parse_csv("raw_utara.csv", "1_North.csv")

['N', '00', '54', '34.28', '']
E,108º,59',20.42"
['E', '108', '59', '20.42', '']
['N', '00', '54', '25.71', '']
E,108º,59',12.34"
['E', '108', '59', '12.34', '']
['N', '00', '54', '26.50', '']
E,108º,59',11.82"
['E', '108', '59', '11.82', '']
['N', '00', '54', '09.77', '']
E,108º,58',52.89"
['E', '108', '58', '52.89', '']
['N', '00', '54', '30.42', '']
E,108º,58',58.57"
['E', '108', '58', '58.57', '']
['N', '00', '54', '30.76', '']
E,108º,58',57.58"
['E', '108', '58', '57.58', '']
['N', '00', '54', '16.87', '']
E,108º,58',53.90"
['E', '108', '58', '53.90', '']
['N', '00', '53', '59.20', '']
E,108º,58',25.45"
['E', '108', '58', '25.45', '']
['N', '00', '53', '48.38', '']
E,108º,58',20.04"
['E', '108', '58', '20.04', '']
['N', '00', '54', '33.01', '']
E,108º,59',06.68"
['E', '108', '59', '06.68', '']
['N', '00', '54', '48.39', '']
E,108º,59',11.43"
['E', '108', '59', '11.43', '']
['N', '00', '54', '34.59', '']
E,108º,59',01.83"
['E', '108', '59', '01.83', '']
['N', '00', '54', '52.16', '

ValueError: could not convert string to float: '00.xx'

In [92]:
# Compile changes into a single csv

import csv, json

array = [
    ["1_West.csv", "Barat (West)", "w"],
    ["1_Central.csv", "Tengah (Central)", "c"],
    ["1_South.csv", "Selatan (South)", "s"],
    ["1_East.csv", "Timur (East)", "e"],
    ["1_North.csv", "Utara (North)", "n"]
]

outlist = []
image = {}
with open("./Num_images.csv") as csvfile:
    reader = csv.reader(csvfile)
    
    next(reader, None)
    
    for line in reader:
        image[line[0]] = {
            "num_images": line[2]
        }

for entry in array:
    with open(entry[0]) as csvfile:
        reader = csv.reader(csvfile)
        
        idnum = 1
        
        next(reader, None)
        
        for line in reader:
            
            id = entry[2] + str(idnum)
            print(id)
            print(line)
            
            id = entry[1][0] + str(idnum)
            outlist.append({
                "name_indo": line[0],
                "name_hanzi": line[1],
                "name_pinyin": line[2],
                "road": line[3],
                "gps_lat": line[4],
                "gps_lon": line[5],
                "address": line[6],
                "ref_no": line[7],
                "id": id,
                "num_images": image[line[7]]["num_images"],
                "region": entry[1]
            })
            
            idnum += 1

keys = outlist[0].keys()
with open("2_combined.csv", 'w', newline='', encoding='utf-8')  as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(outlist)

w1
['VIHARA TRIDHARMA BUMI RAYA', '山口洋中央伯公廟', 'Shan Kou Yang Zhong Yang Bo Gong Miao', 'PUSAT KOTA SINGKAWANG', '0.9095222222222222', '108.98900555555555', 'Jl. Sejahtera No.1, RT 48/06 Kelurahan Melayu', '1']
w2
['VIHARA TRIDHARMA ADI MOKKHA', '華嶽宮/華光帝(華都街)', 'Hua Yue Gong / Huang Guang Di', 'JL. P. DIPONEGORO', '0.9071416666666667', '108.98676111111111', 'Jl. P. Diponegoro No.1, RT 02/01 Kelurahan Pasiran', '2']
w3
['VIHARA TRIDHARMA KALAMA KWANTI', '協天官/關帝(華都街)', 'Xie Tian Gong / Guan Di', 'JL. P. DIPONEGORO', '0.9073611111111112', '108.98661666666666', 'Jl. P. Diponegoro No.1, RT 02/01 Kelurahan Pasiran', '3']
w4
['CETIYA FA SAN THAI CIONG', '華山大将廟(吧西蘭)', 'Hua Shan Da Jiang Miao', 'JL. P. DIPONEGORO', '0.9027138888888889', '108.98135833333333', 'Jl. P. Diponegoro Gg. Tujuh Belas, RT 56/18 Kelurahan Pasiran', '']
w5
['VIHARA DHARMA BUDDHA MAITREYA', '彌勒大道慧發佛堂', 'Mi Le Da Dao Hui Fa Fo Tang', 'JL. SM. TSJAFIOEDDIN', '0.90845', '108.98293611111112', 'Jl. SM. Tsjafioeddin No.32, RT 14/

In [27]:
def csv_to_geojson(infile, outfile):

    import csv, json
    from geojson import Feature, FeatureCollection, Point

    features = []
    with open(infile, newline='') as csvfile:
        
        reader = csv.reader(csvfile, delimiter=',')
        next(reader, None)
        
        for name_indo, name_hanzi, name_pinyin, neighbourhood, gps_lat, gps_lon, address, ref_no,region, id in reader:
            latitude, longitude = map(float, (gps_lat, gps_lon))
                        
            features.append(
                Feature(
                    geometry = Point((longitude, latitude)),
                    properties = {
                        'name_indo': name_indo,
                        'name_hanzi': name_hanzi,
                        'name_pinyin': name_pinyin,
                        'neighbourhood': neighbourhood,
                        'gps_lat': gps_lat,
                        'gps_lon': gps_lon,
                        'address': address,
                        "ref_no": ref_no,
                        "region": region,
                        "id": id
                    }
                )
            )
            
        collection = FeatureCollection(features)
        
        with open(outfile, "w") as f:
            f.write('%s' % collection)

In [28]:
csv_to_geojson("2_combined.csv", "3_combined.geojson")

# Altar 

In [8]:
import csv, re

def parse_altar_csv (infile, outfile):

    with open(infile) as csvfile:
        reader = csv.reader(csvfile)

        next(reader, None)

        outlist = []
        buffer= {
                "UID": "",
                "altar_Name": "",
                "address": "",
                "village": "",
                "gps_lat": "",
                "gps_lon": "",
                "region": "",
                "num_images": "",
                "photos": "",
                "photo_ref": "",
                "pantak": ""
            }

        for row in reader:
            address = ""
            
            if row[0]:
                buffer["UID"] = row[0]
                buffer["altar_Name"] = row[1]
                buffer["address"] = row[5]
                
                gps_lat = "N," + row[12] + "º" + row[13] + "'" + row[14] + "." + row[15] + "\""
                gps_lat = parse_dms(gps_lat)
                
                buffer["gps_lat"] = gps_lat
                buffer["photo_ref"] = row[17]
                
                if int(row[17]) <= 50:
                    buffer["photos"] = "Y"
                else:
                    buffer["photos"] = "N"
                
                buffer["region"] = row[18]
                buffer["village"] = row[19]
                buffer["num_images"] = "1"
                
            else:
                
                if row[6] != "":
                    buffer["address"] = buffer["address"] + row[5] + row[6]
                    
                buffer["address"] = buffer["address"] + "," + row[7] + row[8]
                gps_lon = "E," + row[12] + "º" + row[13] + "'" + row[14] + "." + row[15] + "\""
                gps_lon = parse_dms(gps_lon)
                buffer["gps_lon"] = gps_lon
                
                buffer["pantak"] = row[17]
                
                outlist.append(buffer)
                buffer= {
                "UID": "",
                "altar_Name": "",
                "address": "",
                "village": "",
                "gps_lat": "",
                "gps_lon": "",
                "region": "",
                "photos": "",
                "num_images": "",
                "photo_ref": "",
                "pantak": ""
            }
                
    keys = outlist[0].keys()
    with open(outfile, 'w', newline='', encoding='utf-8')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(outlist)

In [10]:
parse_altar_csv("./altar_raw.csv", "./altars.csv")